## Installing Required Libraries

We are installing essential libraries for our project:

- **datasets**: For loading and preprocessing datasets.
- **evaluate**: For evaluating machine learning models.
- **accelerate**: For speeding up training and inference.
- **transformers[torch]**: For using pre-trained NLP models with PyTorch.

### Installation Command

In [1]:
! pip install --quiet datasets evaluate accelerate transformers[torch]

## Loading the Amharic News Text Classification Dataset and Preprocessing the data.

In [1]:
from datasets import load_dataset
all_dataset = load_dataset('csv', data_files='data/Amharic News Dataset.csv')
all_dataset

c:\Users\fikre\AppData\Local\Programs\Python\Python311\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Generating train split: 51483 examples [00:03, 12890.76 examples/s]


DatasetDict({
    train: Dataset({
        features: ['headline', 'category', 'date', 'views', 'article', 'link'],
        num_rows: 51483
    })
})

In [2]:
import re
# method to normalize character level missmatch such as ጸሀይ and ፀሐይ
def normalize_char_level_missmatch(input_token):
    rep1 = re.sub('[ሃኅኃሐሓኻ]', 'ሀ', input_token)
    rep2 = re.sub('[ሑኁዅ]', 'ሁ', rep1)
    rep3 = re.sub('[ኂሒኺ]', 'ሂ', rep2)
    rep4 = re.sub('[ኌሔዄ]', 'ሄ', rep3)
    rep5 = re.sub('[ሕኅ]', 'ህ', rep4)
    rep6 = re.sub('[ኆሖኾ]', 'ሆ', rep5)
    rep7 = re.sub('[ሠ]', 'ሰ', rep6)
    rep8 = re.sub('[ሡ]', 'ሱ', rep7)
    rep9 = re.sub('[ሢ]', 'ሲ', rep8)
    rep10 = re.sub('[ሣ]', 'ሳ', rep9)
    rep11 = re.sub('[ሤ]', 'ሴ', rep10)
    rep12 = re.sub('[ሥ]', 'ስ', rep11)
    rep13 = re.sub('[ሦ]', 'ሶ', rep12)
    rep14 = re.sub('[ዓኣዐ]', 'አ', rep13)
    rep15 = re.sub('[ዑ]', 'ኡ', rep14)
    rep16 = re.sub('[ዒ]', 'ኢ', rep15)
    rep17 = re.sub('[ዔ]', 'ኤ', rep16)
    rep18 = re.sub('[ዕ]', 'እ', rep17)
    rep19 = re.sub('[ዖ]', 'ኦ', rep18)
    rep20 = re.sub('[ጸ]', 'ፀ', rep19)
    rep21 = re.sub('[ጹ]', 'ፁ', rep20)
    rep22 = re.sub('[ጺ]', 'ፂ', rep21)
    rep23 = re.sub('[ጻ]', 'ፃ', rep22)
    rep24 = re.sub('[ጼ]', 'ፄ', rep23)
    rep25 = re.sub('[ጽ]', 'ፅ', rep24)
    rep26 = re.sub('[ጾ]', 'ፆ', rep25)
    # Normalizing words with Labialized Amharic characters such as በልቱዋል or  በልቱአል to  በልቷል
    rep27 = re.sub('(ሉ[ዋአ])', 'ሏ', rep26)
    rep28 = re.sub('(ሙ[ዋአ])', 'ሟ', rep27)
    rep29 = re.sub('(ቱ[ዋአ])', 'ቷ', rep28)
    rep30 = re.sub('(ሩ[ዋአ])', 'ሯ', rep29)
    rep31 = re.sub('(ሱ[ዋአ])', 'ሷ', rep30)
    rep32 = re.sub('(ሹ[ዋአ])', 'ሿ', rep31)
    rep33 = re.sub('(ቁ[ዋአ])', 'ቋ', rep32)
    rep34 = re.sub('(ቡ[ዋአ])', 'ቧ', rep33)
    rep35 = re.sub('(ቹ[ዋአ])', 'ቿ', rep34)
    rep36 = re.sub('(ሁ[ዋአ])', 'ኋ', rep35)
    rep37 = re.sub('(ኑ[ዋአ])', 'ኗ', rep36)
    rep38 = re.sub('(ኙ[ዋአ])', 'ኟ', rep37)
    rep39 = re.sub('(ኩ[ዋአ])', 'ኳ', rep38)
    rep40 = re.sub('(ዙ[ዋአ])', 'ዟ', rep39)
    rep41 = re.sub('(ጉ[ዋአ])', 'ጓ', rep40)
    rep42 = re.sub('(ደ[ዋአ])', 'ዷ', rep41)
    rep43 = re.sub('(ጡ[ዋአ])', 'ጧ', rep42)
    rep44 = re.sub('(ጩ[ዋአ])', 'ጯ', rep43)
    rep45 = re.sub('(ጹ[ዋአ])', 'ጿ', rep44)
    rep46 = re.sub('(ፉ[ዋአ])', 'ፏ', rep45)
    rep47 = re.sub('[ቊ]', 'ቁ', rep46)  # ቁ can be written as ቊ
    rep48 = re.sub('[ኵ]', 'ኩ', rep47)  # ኩ can be also written as ኵ
    return rep48

# Splitting the dataset into train and test
dataset = all_dataset['train'].train_test_split(test_size=0.2, seed=42) 
# Filtering the dataset to remove the rows that have None in the category column
dataset = dataset.filter(lambda x: x['category'] is not None)
# Normalizing the category column
dataset = dataset.map(lambda x: {'category': normalize_char_level_missmatch(x['category'])})
print(dataset)

# Getting the unique categories that will be used as labels for the model.
categories = list(sorted(set(dataset['train']['category'])))
print("Categories -" ,categories)

Map: 100%|██████████| 10297/10297 [00:01<00:00, 9091.71 examples/s]

DatasetDict({
    train: Dataset({
        features: ['headline', 'category', 'date', 'views', 'article', 'link'],
        num_rows: 41185
    })
    test: Dataset({
        features: ['headline', 'category', 'date', 'views', 'article', 'link'],
        num_rows: 10297
    })
})
Categories - ['ሀገር አቀፍ ዜና', 'መዝናኛ', 'ስፖርት', 'ቢዝነስ', 'አለም አቀፍ ዜና', 'ፖለቲካ']


In [3]:
from transformers import AutoTokenizer, DataCollatorWithPadding

model_checkpoint = "FacebookAI/xlm-roberta-base"
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

# Tokenize the dataset using a lambda function
tokenized_dataset = dataset.map(lambda item: tokenizer(
    item['article'], truncation=True, max_length=512), batched=True)

# Add numerical labels to the tokenized dataset
def add_labels(example):
    example['labels'] = [categories.index(item) for item in example['category']]
    return example
tokenized_dataset = tokenized_dataset.map(add_labels, batched= True)

print(dataset)
print(tokenized_dataset) # Tokenizer added two fields: input_ids and attention_mask

# Use a data collator to apply dynamic padding
data_collator = DataCollatorWithPadding(
    tokenizer=tokenizer, return_tensors='pt')

Map: 100%|██████████| 10297/10297 [00:00<00:00, 62813.60 examples/s]


DatasetDict({
    train: Dataset({
        features: ['headline', 'category', 'date', 'views', 'article', 'link'],
        num_rows: 41185
    })
    test: Dataset({
        features: ['headline', 'category', 'date', 'views', 'article', 'link'],
        num_rows: 10297
    })
})
DatasetDict({
    train: Dataset({
        features: ['headline', 'category', 'date', 'views', 'article', 'link', 'input_ids', 'attention_mask', 'labels'],
        num_rows: 41185
    })
    test: Dataset({
        features: ['headline', 'category', 'date', 'views', 'article', 'link', 'input_ids', 'attention_mask', 'labels'],
        num_rows: 10297
    })
})


In [ ]:
for y in tokenized_dataset['train']:
    original_article = y['article']
    tokenized_input_ids = y['input_ids']
    
    decoded_article = tokenizer.decode(
        tokenized_input_ids, skip_special_tokens=True)

    print("Original Article:")
    print(original_article)
    print("Tokenized Input IDs:")
    print(tokenized_input_ids)
    print("Decoded Article")
    print(decoded_article)
    print("Length of Orginal Article: ", len(original_article))
    print("Length of Decoded Article: ", len(decoded_article))
    print("Length of Input Ids: ", len(tokenized_input_ids))
    break

In [5]:
import torch
from transformers import AutoModelForSequenceClassification, TrainingArguments
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
import numpy as np

# We are using the XLM ROberat. We are configuring it for text classification task
fine_tuned_model = AutoModelForSequenceClassification.from_pretrained(
    model_checkpoint,
    num_labels=len(categories),
    id2label={i: lbl for i, lbl in enumerate(categories)},
    label2id={lbl: i for i, lbl in enumerate(categories)},
    device_map="cuda"
)

# Move the model to GPU if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
fine_tuned_model.to(device)

# Print the device to confirm
print(f"Model loaded on: {device}")

# Seting up the fine tunning parameter
batch_size = 16
gradient_accumulation_steps = 4
epochs = 5

training_args = TrainingArguments(
    output_dir=model_checkpoint+"-finetuned",
    learning_rate=5e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    gradient_accumulation_steps=gradient_accumulation_steps,
    num_train_epochs=epochs,
    weight_decay=0.1,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    logging_strategy="epoch",
    load_best_model_at_end=True,
    metric_for_best_model="f1",
    fp16=True,
    seed=42,
)

# Define the function to compute evaluation metrics
def measure_performance(eval_preds):
    logits, labels = eval_preds
    predictions = np.argmax(logits, axis=-1)
    accuracy = accuracy_score(labels, predictions)
    precision = precision_score(labels, predictions, average='macro')
    recall = recall_score(labels, predictions, average='macro')
    f1 = f1_score(labels, predictions, average='macro')

    return { "Accuracy": accuracy, "Precision": precision, "Recall": recall, "F1": f1}

Some weights of XLMRobertaForSequenceClassification were not initialized from the model checkpoint at FacebookAI/xlm-roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Model loaded on: cuda


In [6]:
from transformers import Trainer

trainer = Trainer(
    model=fine_tuned_model,
    args=training_args,
    train_dataset=tokenized_dataset["train"],
    eval_dataset=tokenized_dataset["test"],
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=measure_performance,
)
trainer.train(resume_from_checkpoint=True)

C:\Users\fikre\AppData\Local\Temp\ipykernel_21760\306540567.py:3: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
c:\Users\fikre\AppData\Local\Programs\Python\Python311\Lib\site-packages\transformers\trainer.py:3354: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommen

{'train_runtime': 1.4089, 'train_samples_per_second': 146159.702, 'train_steps_per_second': 2281.915, 'train_loss': 0.0, 'epoch': 5.0}


TrainOutput(global_step=3215, training_loss=0.0, metrics={'train_runtime': 1.4089, 'train_samples_per_second': 146159.702, 'train_steps_per_second': 2281.915, 'total_flos': 5.415114548373984e+16, 'train_loss': 0.0, 'epoch': 4.9984466019417475})

In [12]:
from transformers import AutoModelForSequenceClassification, AutoTokenizer
import torch

fine_tuned_checkpoint = "FacebookAI/xlm-roberta-base-finetuned/checkpoint-3215"
tokenizer = AutoTokenizer.from_pretrained(fine_tuned_checkpoint)
fine_tuned_model = AutoModelForSequenceClassification.from_pretrained(
    fine_tuned_checkpoint)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
fine_tuned_model.to(device)

XLMRobertaForSequenceClassification(
  (roberta): XLMRobertaModel(
    (embeddings): XLMRobertaEmbeddings(
      (word_embeddings): Embedding(250002, 768, padding_idx=1)
      (position_embeddings): Embedding(514, 768, padding_idx=1)
      (token_type_embeddings): Embedding(1, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): XLMRobertaEncoder(
      (layer): ModuleList(
        (0-11): 12 x XLMRobertaLayer(
          (attention): XLMRobertaAttention(
            (self): XLMRobertaSdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): XLMRobertaSelfOutput(
              (dense): Linear(in_features=768, out_features=

In [13]:
from sklearn.metrics import precision_recall_fscore_support

# Get the true labels and predictions
true_labels = []
predictions = []

for batch in tokenized_dataset["test"]:
    inputs = {key: torch.tensor([value]).to(device) for key, value in batch.items() if key in tokenizer.model_input_names}
    with torch.no_grad():
        outputs = fine_tuned_model(**inputs)
    logits = outputs.logits
    predicted_label = torch.argmax(logits, dim=-1).item()
    true_labels.append(batch['labels'])
    predictions.append(predicted_label)

# Calculate precision, recall, and f1 for each category
precision, recall, f1, _ = precision_recall_fscore_support(true_labels, predictions, average=None, labels=range(len(categories)))

# Calculate the average precision, recall, and f1
average_precision = precision.mean()
average_recall = recall.mean()
average_f1 = f1.mean()

# Print the evaluation results
print("Evaluation Results:")
print(f"Average Precision: {average_precision}")
print(f"Average Recall: {average_recall}")
print(f"Average F1: {average_f1}")

print("\nPrecision, Recall, and F1 for each category:")
for i, category in enumerate(categories):
    print(f"Category: {category}")
    print(f"Precision: {precision[i]}")
    print(f"Recall: {recall[i]}")
    print(f"F1: {f1[i]}")
    print()

print("Average Precision:", average_precision)
print("Average Recall:", average_recall)
print("Average F1:", average_f1)

Evaluation Results:
Average Precision: 0.866959995741114
Average Recall: 0.8557428605739928
Average F1: 0.8601660349030417

Precision, Recall, and F1 for each category:
Category: ሀገር አቀፍ ዜና
Precision: 0.9113142292490118
Recall: 0.8848644758934996
F1: 0.8978946087379822

Category: መዝናኛ
Precision: 0.8681318681318682
Recall: 0.7314814814814815
F1: 0.7939698492462312

Category: ስፖርት
Precision: 0.9731800766283525
Recall: 0.9931573802541545
F1: 0.9830672472181906

Category: ቢዝነስ
Precision: 0.7129186602870813
Recall: 0.7631241997439181
F1: 0.7371675943104514

Category: አለም አቀፍ ዜና
Precision: 0.8984198645598194
Recall: 0.9191685912240185
F1: 0.908675799086758

Category: ፖለቲካ
Precision: 0.8377952755905512
Recall: 0.8426610348468849
F1: 0.8402211108186365

Average Precision: 0.866959995741114
Average Recall: 0.8557428605739928
Average F1: 0.8601660349030417


In [14]:
# Sample Prediction

input_paragraph = """
የማንቸስተር ዩናይትዱ አሰልጣኝ ሩበን አሞሪም ቡድናቸው በክለቡ 147 ዓመታት ታሪክ "በጣም ደካማው" ሳይሆን እንደማይቀር ገለጸ።

ቡድኑ በብራይተን 3 ለ 1 የተሸነፈበት ጨዋታ በሜዳው ካደረጋቸው ያለፉት አምስት የፕሪሚር ሊግ መርሐ ግብሮች በአራቱ የተሸነፈበት ሆኗል።

አሞሪም ኤሪክ ቴን ሃግን ተከቶ ቡድኑን ከተረከበ በኋላ ፖርቹጋላዊው ካለፉት 11 ጨዋታዎች 11 ነጥብ ብቻ ሰብስቧል።

ዩናይትድ ከመውረድ ቀጠና በ10 ነጥብ ርቆ 13ኛ ደረጃ ላይ ተቀምጧል።

"ካለፉት 10 የሊጉ ጨዋታዎች ሁለቱን ነው ያሸነፍነው። ይህ ለደጋፊዎቹም ሆነ ለእኔ ምን ማለት እንደሆነ አስቡት" ብሏል።

"አዲሱ አሰልጣኝ ካለፈው አሰልጣኝ በበለጠ እየተሸነፈ ነው። ይህንን አውቃለሁ።"

"በማንቸስተር ዩናይትድ ታሪክ ደካማው ቡድን ሳንሆን አንቀርም። እናንተ [መገናኛ ብዙሃን] ርዕስ እንደምትፈልጉ ባውቅም እውነቱን ተናግሬ መቀየር ይኖርብኛል። ይኸው ርዕስ ሰጠዋችሁ" ሲል ተናግሯል።

በብራይተን ጎል ካስተናገደ በኋላ ዩናይትድ አቻ ለመሆን በቅቶ ነበር።

የብሩኖ ፈርናንዴስ ፍጹም ቅጣት ምት ቡድኑን ያነቃቃዋል ቢባልም ይበልጥ ተበልጦ ታይቷል።

ካኦሩ ሚቶማ ቡድኑን ቀዳሚ ሲያደርግ ግብ ጠባቂው አንድሬ ኦናና የፈጸመውን ስህተት ተጠቅሞ ጂዮርጂኖ ረተር ለቡድኑ ሦስተኛዋን ጎል ከመረብ አሳርፏል።
"""
# Tokenize the input paragraph
inputs = tokenizer(input_paragraph, padding=True,
                   truncation=True, return_tensors="pt", max_length=512)

inputs = {key: value.to(device) for key, value in inputs.items()}

# Make predictions
fine_tuned_model.eval()
with torch.no_grad():
    outputs = fine_tuned_model(**inputs)

logits = outputs.logits
predictions = torch.argmax(logits, dim=-1)

# Map predictions to labels
id2label = fine_tuned_model.config.id2label
predicted_label = id2label[predictions.item()]

# Print the prediction
print(f"Predicted Label: {predicted_label}")

Predicted Label: ስፖርት
